## Imports

In [3]:
import torch, torchvision
import numpy as np

In [4]:
MNIST = torchvision.datasets.MNIST("./data", train=True, download=True)

x = MNIST.data.flatten(1).int()
y = MNIST.targets.int()

print(x.shape, y.shape)

x = x.numpy()
y = y.numpy()

print((x > 0).sum(), (x == 0).sum())


#np.savetxt("data/mnist_x.csv", x, delimiter=",", fmt="%d")
#np.savetxt("data/mnist_y.csv", y, delimiter=",", fmt="%d")

torch.Size([60000, 784]) torch.Size([60000])
8994156 38045844


In [5]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.linear1 = torch.nn.Linear(input_size, hidden_size)
        self.linear2 = torch.nn.Linear(hidden_size, hidden_size)
        self.linear3 = torch.nn.Linear(hidden_size, hidden_size)
        self.linear4 = torch.nn.Linear(hidden_size, output_size)
        self.act = torch.nn.ReLU()

    def forward(self, x):
        x = self.act(self.linear1(x))
        x = self.act(self.linear2(x))
        x = self.act(self.linear3(x))
        x = self.linear4(x)
        return x

In [6]:
dataset = torch.utils.data.TensorDataset(torch.tensor(x), torch.tensor(y))
train_loader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)

In [7]:
import time
num_epochs = 100
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"

model = NeuralNetwork(784, 300, 10)
model = model.to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

start = time.time()
for epoch in range(num_epochs):
    b_x, b_y = next(iter(train_loader))
    b_x = b_x.float().to(device)
    b_y = b_y.long().to(device)
    output = model(b_x)
    loss = loss_fn(output, b_y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {(output.argmax(1) == b_y).float().mean():.4f}")

print(f"Time: {time.time() - start:.2f} seconds")


Epoch 10/100, Loss: 1.7722, Accuracy: 0.4100
Epoch 20/100, Loss: 1.2770, Accuracy: 0.6000
Epoch 30/100, Loss: 0.9039, Accuracy: 0.7200
Epoch 40/100, Loss: 0.8120, Accuracy: 0.7600
Epoch 50/100, Loss: 0.7851, Accuracy: 0.7500
Epoch 60/100, Loss: 0.6332, Accuracy: 0.8100
Epoch 70/100, Loss: 0.5385, Accuracy: 0.7800
Epoch 80/100, Loss: 0.4535, Accuracy: 0.8500
Epoch 90/100, Loss: 0.7426, Accuracy: 0.7500
Epoch 100/100, Loss: 0.6234, Accuracy: 0.7800
Time: 1.98 seconds
